In [1]:
from ML4transients.training import PytorchDataset, CustomCNN, get_trainer, get_loss_function

2025-07-29 12:29:48.965245: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-07-29 12:29:48.982522: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1753784989.003781 3096967 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1753784989.010324 3096967 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1753784989.027074 3096967 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking 

In [2]:
datasets = PytorchDataset.create_splits('/sps/lsst/groups/transients/HSC/fouchez/raphael/rc2_norm', random_state=42)

Building sample index...
Creating splits from 57905 samples...
Loading 40533 cutouts...
Loading 5791 cutouts...
Loading 11581 cutouts...


In [3]:
train_dataset = datasets['train'] 
test_dataset = datasets['test'] 
val_dataset = datasets['val'] 



In [5]:
from torch.utils.data import DataLoader

config = {
    "epochs": 2,
    "learning_rate": 0.001,
    "batch_size": 128,
    "num_iter_per_epoch": 400,
    "epoch_decay_start": 80,
    "num_workers": 0,
    "output_dir":"../saved/training_test",
    "model_params": {
        "input_shape": [30, 30, 1],
        "num_classes": 2,
        "filters_1": 32,
        "filters_2": 64,
        "dropout_1": 0.25,
        "dropout_2": 0.25,
        "dropout_3": 0.5,
        "units": 128
    }
   
}

In [6]:
train_loader = DataLoader(
    train_dataset, 
    batch_size=config['batch_size'],  # 128 from your config
    shuffle=True,
    num_workers=config['num_workers']  # 4 from your config
)
test_loader = DataLoader(
    test_dataset, 
    batch_size=config['batch_size'], 
    shuffle=False,
    num_workers=config['num_workers']
)

val_loader = DataLoader(
    val_dataset, 
    batch_size=config['batch_size'], 
    shuffle=False,
    num_workers=config['num_workers']
)
trainer = get_trainer("standard", config)




TensorBoard logging to: runs/experiment
Run 'tensorboard --logdir=runs' to view logs


In [7]:
trainer.train_one_epoch(1, train_loader)

{'loss': 0.24656122683162568, 'accuracy': 0.8974909333135963}

In [9]:
best_acc = trainer.fit(train_loader, test_loader, val_loader)


Epoch 1/2 - Train: {'loss': 0.15585329724411107, 'accuracy': 0.9421705770606666} - Test: {'accuracy': 0.9445643726793886} - Val: {'accuracy': 0.9475047487480573}
Epoch 2/2 - Train: {'loss': 0.14369315182796036, 'accuracy': 0.9447610588902869} - Test: {'accuracy': 0.94387358604611} - Val: {'accuracy': 0.9495769297185288}


In [8]:
import torch
trainer = get_trainer("standard", config)
trainer.model.load_state_dict(torch.load('/sps/lsst/users/rbonnetguerrini/ML4transients/model_best.pth', map_location=torch.device('cpu')))

trainer

TensorBoard logging to: runs/experiment
Run 'tensorboard --logdir=runs' to view logs


In [9]:
from torch.utils.data import DataLoader

inference_loader = DataLoader(
    val_dataset,  # Replace with your dataset
    batch_size=1,  # Batch size of 1 for inference
    shuffle=False,
    num_workers=0
)

In [10]:
trainer.model.eval()  # Set the model to evaluation mode
predictions = []

with torch.no_grad():  # Disable gradient computation
    for images, labels, _ in inference_loader:  # Replace with your DataLoader
        images = images.to(trainer.device)
        outputs = trainer.model(images)
        preds = (torch.sigmoid(outputs.squeeze()) > 0.5).float()  # Binary classification threshold
        predictions.append(preds.cpu().numpy())

print(predictions)

[array(0., dtype=float32), array(0., dtype=float32), array(0., dtype=float32), array(0., dtype=float32), array(0., dtype=float32), array(0., dtype=float32), array(1., dtype=float32), array(0., dtype=float32), array(0., dtype=float32), array(0., dtype=float32), array(0., dtype=float32), array(0., dtype=float32), array(0., dtype=float32), array(0., dtype=float32), array(0., dtype=float32), array(0., dtype=float32), array(0., dtype=float32), array(1., dtype=float32), array(0., dtype=float32), array(0., dtype=float32), array(1., dtype=float32), array(0., dtype=float32), array(1., dtype=float32), array(0., dtype=float32), array(0., dtype=float32), array(0., dtype=float32), array(0., dtype=float32), array(1., dtype=float32), array(0., dtype=float32), array(0., dtype=float32), array(1., dtype=float32), array(0., dtype=float32), array(0., dtype=float32), array(0., dtype=float32), array(1., dtype=float32), array(0., dtype=float32), array(0., dtype=float32), array(0., dtype=float32), array(0., d